In [4]:
%pip install langchain_community openai elasticsearch tiktoken langchain_elasticsearch langchain_openai cohere python-dotenv xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 7.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [233 lines of output]
      running bdist_wheel
      /Users/jeongmin/PycharmProjects/python-skills-bootcamp/.venv/lib/python3.9/site-packages/torch/utils/cpp_extension.py:499: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-universal2-3.9
      creating build/lib.macosx-10.9-universal2-3.9/xformers
      copying xformers/_deprecation_warning.py -> build/lib.macosx-10.9-universal2-3.9/xformers
      copying xformers/attn_bias_utils.py -> build/lib.macosx-10.9-universal2-3.9/xformers


# Elasticsearch: Dense Search 

DenseSearch 는 Embedding 을 이용한 검색임. 

다음과 같이 인덱스에 dense_vector 필드가 있어야한다. 그리고 Hybrid Search 를 하려면 keyword 타입의 필드도 있어야함. 

```json 
PUT my-index
{
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "dense_vector",
        "dims": 3
      },
      "my_text" : {
        "type" : "keyword"
      }
    }
  }
} 

PUT my-index/_doc/1
{
  "my_text" : "text1",
  "my_vector" : [0.5, 10, 6]
}
```

문서가 있다면 Vector Type 의 임베딩을 만드는 과정이 필요하다. 




In [21]:
import pprint
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings.openai import OpenAIEmbeddings
import json
import ssl 
import cohere
import os 
from dotenv import load_dotenv

load_dotenv(override=True)

COHERE_API_KEY= ""
COHERE_API_KEY2 = ""

co = cohere.Client(COHERE_API_KEY2)

CLOUD_ID = "35e14e0f2e12492e832ab6a800d0b2c1:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGNmYjVlMzZkMzhjMjRhODA4NTA4NzgyNDRmOTNjMjg0JGYyNDFiNzhiMDY5MjQ4NzFhY2RkZmFiOWMyMzg5MTdl"
API_KEY = ""

index = "hybrid_search_index"

ES_HOST = "https://localhost:9200"
ES_USER = "elastic"
ES_PASSWORD = "DdgnuaPpFlewuaq_R7BD"
ES_API_KEY = ""
CA_CERT_PATH = "/Users/jeongmin/http_ca.crt"  # CA 인증서 경로

api_key_header = "ApiKey " + ES_API_KEY
headers={"Authorization": api_key_header, "Content-Type": "application/json"}

# Create an SSL context
ssl_context = ssl.create_default_context(cafile=CA_CERT_PATH)
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# openai text-embedding-ada-002 model dimentions is 1536 
# use embed_query() meyhod 
embeddings = OpenAIEmbeddings()

# sentencetransfomer model dimentions is 384
# use encode() method 
transfomer = SentenceTransformer('all-MiniLM-L6-v2')

class ElasticsearchIndexer: 
    def __init__(self, index_name="hybrid_search_index", model=transfomer):
        self.model = model
        self.es = Elasticsearch(
            [ES_HOST],
            basic_auth=(ES_USER, ES_PASSWORD),
            ssl_context=ssl_context     
        )
        self.index = index_name
        print('Created ElasticsearchIndexer! successfully')
        
    def insert_documents(self, documents):
        operations = []
        for document in documents:
            operations.append({'index': {'_index': self.index}})
            
            doc_with_vectors = document.copy()
            doc_with_vectors.update({
                'name_vector': self.get_embedding(document.get('name', '')),
                'summary_vector': self.get_embedding(document.get('summary', '')),
                'content_vector': self.get_embedding(document.get('content', ''))
            })
            
            operations.append(doc_with_vectors)

        return self.es.bulk(operations=operations)

    def reindex(self):
        self.create_index()
        with open('data.json', 'rt') as f:
            documents = json.loads(f.read())
        return self.insert_documents(documents)

    def create_index(self):
        self.es.indices.delete(index=index, ignore_unavailable=True)
        self.es.indices.create(index=index, mappings={
            "properties": {
                "name": {
                    "type": "text",
                    "fields": {
                    "keyword": {
                        "type": "keyword"
                    }
                    }
                },
                "name_vector": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index": True,
                    "similarity": "cosine"
                },
                "url": {
                    "type": "keyword"
                },
                "summary": {
                    "type": "text",
                    "fields": {
                    "keyword": {
                        "type": "keyword"
                    }
                    }
                },
                "summary_vector": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index": True,
                    "similarity": "cosine"
                },
                "content": {
                    "type": "text",
                    "fields": {
                    "keyword": {
                        "type": "keyword"
                    }
                    }
                },
                "content_vector": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index": True,
                    "similarity": "cosine"
                },
                "created_on": {
                    "type": "date"
                },
                "category": {
                    "type": "keyword"
                },
                "rolePermissions": {
                    "type": "keyword"
                }
            }
        })

    def get_embedding(self, text):
        return self.model.encode(text)



class Search:
    def __init__(self, model=transfomer, cohere=co):
        self.model = model
        self.es = Elasticsearch(
            [ES_HOST],
            basic_auth=(ES_USER, ES_PASSWORD),
            ssl_context=ssl_context     
        )
        self.co = cohere
        print('Created to Elasticsearch! successfully')

    
    def get_embedding(self, text):
        return self.model.encode(text)

    
    def baseline_search(self, query, from_=0, size=5):
        results = self.es.search(index="hybrid_search_index", body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["name", "summary", "content"]
                }
            }
        })
        
        return [hit['_source'] for hit in results['hits']['hits']]

    def hybrid_search_default(self, query, from_=0, size=5):
        search_query = {
            'must': [
                {
                    'multi_match': {
                        'query': query,
                        'fields': ['name', 'summary', 'content'],
                    }
                },
                {
                    'bool': {
                        'must': [
                            {'exists': {'field': 'name_vector'}},
                            {'exists': {'field': 'summary_vector'}},
                            {'exists': {'field': 'content_vector'}}
                        ]
                    }
                }
            ]
        }

        results = self.es.search(
            index='hybrid_search_index',
            query={
                'bool': search_query
            },
            knn=[
                {
                    'field': 'name_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 50,
                },
                {
                    'field': 'summary_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 50,
                },
                {
                    'field': 'content_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 50,
                }
            ],
            # through non-compliance license issue, we cannot use this feature. if you want to use this feature, you should use the elastic cloud. 
            # rank={
            #     'rrf': {}
            # },
            aggs={
                'year-agg': {
                    'date_histogram': {
                        'field': 'updated_at',
                        'calendar_interval': 'year',
                        'format': 'yyyy',
                    },
                },
            },
            size=size,
            from_=from_,
        )

        return [hit['_source'] for hit in results['hits']['hits']]

    # add reranking feature
    # adjust num_candidates parameter
    def hybrid_search_v0(self, query, from_=0, size=5, rerank_top_n=5):
        search_query = {
            'must': [
                {
                    'multi_match': {
                        'query': query,
                        'fields': ['name', 'summary', 'content'],
                    }
                },
                {
                    'bool': {
                        'must': [
                            {'exists': {'field': 'name_vector'}},
                            {'exists': {'field': 'summary_vector'}},
                            {'exists': {'field': 'content_vector'}}
                        ]
                    }
                }
            ]
        }

        results = self.es.search(
            index='hybrid_search_index',
            query={
                'bool': search_query
            },
            knn=[
                {
                    'field': 'name_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 20,
                },
                {
                    'field': 'summary_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 20,
                },
                {
                    'field': 'content_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 20,
                }
            ],
            # through non-compliance license issue, we cannot use this feature. if you want to use this feature, you should use the elastic cloud. 
            # rank={
            #     'rrf': {}
            # },
            aggs={
                'year-agg': {
                    'date_histogram': {
                        'field': 'updated_at',
                        'calendar_interval': 'year',
                        'format': 'yyyy',
                    },
                },
            },
            size=size,
            from_=from_,
        )

        hits = results['hits']['hits']
        docs = [{'id': hit['_id'], 'text': hit['_source']['name'] + ' ' + hit['_source']['summary'] + ' ' + hit['_source']['content'], 'name': hit['_source']['name'], 'summary':hit['_source']['summary'], 'content': hit['_source']['content']} for hit in hits]
        
        rerank_results = self.co.rerank(
            model="rerank-english-v3.0", 
            query=query, 
            documents=docs, 
            top_n=rerank_top_n
        )

        reranked_docs = []
        for rerank_result in rerank_results.results:
            index = rerank_result.index
            reranked_docs.append(docs[index])
            
        return reranked_docs
    

    # adjust boost parameter
    def hybrid_search_v1(self, query, from_=0, size=5, rerank_top_n=5):
        search_query = {
            'must': [
                {
                    'multi_match': {
                        'query': query,
                        'fields': ['name', 'summary', 'content'],
                        'boost': 0.3 
                    }
                },
                {
                    'bool': {
                        'must': [
                            {'exists': {'field': 'name_vector'}},
                            {'exists': {'field': 'summary_vector'}},
                            {'exists': {'field': 'content_vector'}}
                        ]
                    }
                }
            ]
        }

        results = self.es.search(
            index='hybrid_search_index',
            query={
                'bool': search_query
            },
            knn=[
                {
                    'field': 'name_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 20,
                    'boost': 0.7 
                },
                {
                    'field': 'summary_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 20,
                    'boost': 0.7
                },
                {
                    'field': 'content_vector',
                    'query_vector': self.get_embedding(query),
                    'k': 10,
                    'num_candidates': 20,
                    'boost': 0.7
                }
            ],
            # through non-compliance license issue, we cannot use this feature. if you want to use this feature, you should use the elastic cloud. 
            # rank={
            #     'rrf': {}
            # },
            aggs={
                'year-agg': {
                    'date_histogram': {
                        'field': 'updated_at',
                        'calendar_interval': 'year',
                        'format': 'yyyy',
                    },
                },
            },
            size=size,
            from_=from_,
        )

        hits = results['hits']['hits']
        docs = [{'id': hit['_id'], 'text': hit['_source']['name'] + ' ' + hit['_source']['summary'] + ' ' + hit['_source']['content'], 'name': hit['_source']['name'], 'summary':hit['_source']['summary'], 'content': hit['_source']['content']} for hit in hits]
        
        rerank_results = self.co.rerank(
            model="rerank-english-v3.0", 
            query=query, 
            documents=docs, 
            top_n=rerank_top_n
        )

        reranked_docs = []
        for rerank_result in rerank_results.results:
            index = rerank_result.index
            reranked_docs.append(docs[index])
            
        return reranked_docs

indexer = ElasticsearchIndexer()
search = Search()

# # Hybrid Search Test 
# results = search.hybrid_search_default("teacher")
# [doc for doc in results]

/Users/jeongmin/PycharmProjects/python-skills-bootcamp/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Created ElasticsearchIndexer! successfully
Created to Elasticsearch! successfully


In [31]:
indexer.reindex()

ObjectApiResponse({'errors': False, 'took': 762, 'items': [{'index': {'_index': 'hybrid_search_index', '_id': '-dY5IpEB7U6atdh9x8o2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'hybrid_search_index', '_id': '-tY5IpEB7U6atdh9x8o2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'hybrid_search_index', '_id': '-9Y5IpEB7U6atdh9x8o2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'hybrid_search_index', '_id': '_NY5IpEB7U6atdh9x8o2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'hybrid_search_index', '_id': '_dY5IpEB7U6atdh9x8o2'

In [32]:
from pprint import pprint

result = search.es.search(index=index, body={
    'query': {
        'match': {
            'summary': 'The Federal Reserve'
        }
    }
})

for key in result.keys():
    print(f"\nKey: {key}")
    pprint(result[key], width=100, depth=2)

result["hits"]


Key: took
112

Key: timed_out
False

Key: _shards
{'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1}

Key: hits
{'hits': [{...}, {...}, {...}, {...}, {...}, {...}, {...}, {...}, {...}, {...}],
 'max_score': 5.5583973,
 'total': {'relation': 'eq', 'value': 18}}


{'total': {'value': 18, 'relation': 'eq'},
 'max_score': 5.5583973,
 'hits': [{'_index': 'hybrid_search_index',
   '_id': 'utY5IpEB7U6atdh9x842',
   '_score': 5.5583973,
   '_source': {'name': 'Reality last education your under.',
    'url': 'http://www.oliver.biz/',
    'summary': 'Cover do star later federal.',
    'content': 'Away similar hair until girl. Actually discover challenge deal. Reflect important notice risk officer save near back. Leave chair bit become.\nSpecific myself fish ask term poor. Every yourself place stay size. Wait against edge teach ability now agreement item.\nClearly old organization director. He southern significant lawyer. Heart heart report mother anyone.\nReach great fish international. About couple continue truth doctor suggest economic. Why perform program I left investment ball.\nStreet special fall either. On possible knowledge same stage. White speech suffer Mr together also around.\nCare court move week positive notice agency. Especially imagine e

In [9]:
from pprint import pprint

def print_search_results(results):
    print(f"Total hits: {results['total']}")
    print("\nSearch Results:")
    for i, hit in enumerate(results['hits'], 1):
        print(f"\n--- Result {i} ---")
        print(f"Score: {hit['_score']}")
        source = hit['_source']
        print(f"Name: {source.get('name', 'N/A')}")
        print(f"Summary: {source.get('summary', 'N/A')[:100]}...")  # 처음 100자만 출력
        print(f"Category: {source.get('category', 'N/A')}")
        print(f"URL: {source.get('url', 'N/A')}")
        print(f"Created on: {source.get('created_on', 'N/A')}")
        print(f"Updated at: {source.get('updated_at', 'N/A')}")
        print(f"Role Permissions: {', '.join(source.get('rolePermissions', []))}")

    if 'aggregations' in results:
        print("\nYear Aggregations:")
        for bucket in results['aggregations']['year-agg']['buckets']:
            print(f"{bucket['key_as_string']}: {bucket['doc_count']} documents")

# 사용 예:
results = search.hybrid_search_v0("teacher")
print_search_results(results)


AttributeError: 'tuple' object has no attribute 'document'

## Search Performance comparison: Baseline Search vs Hybrind 

In [64]:
%pip install pandas 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:

%pip install openai==0.28

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from IPython.display import display, HTML
import json
import os 
import pandas as pd


class SearchEvaluation(BaseModel):
    baseline_analysis: str = Field(description="A brief analysis of the baseline search results")
    hybrid_analysis: str = Field(description="A brief analysis of the hybrid search results")
    comparison: str = Field(description="A comparison of the two sets of results")
    baseline_score: int = Field(description="A score for the baseline results on a scale of 1-10", ge=1, le=10)
    hybrid_score: int = Field(description="A score for the hybrid results on a scale of 1-10", ge=1, le=10)
    recommendation: str = Field(description="A recommendation on which set of results is better and why")


llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
).with_structured_output(SearchEvaluation)

def format_results(results):
    formatted = []
    for i, source in enumerate(results[:5], 1):
        formatted.append(f"Result {i}:\nTitle: {source.get('name', 'N/A')}\nSummary: {source.get('summary', 'N/A')[:200]}...")
    return "\n\n".join(formatted)

def get_chatgpt_evaluation(query, baseline_results, hybrid_results):
    different_results = []
    for i in range(min(5, len(baseline_results), len(hybrid_results))):
        baseline_name = baseline_results[i].get('name', 'N/A')
        hybrid_name = hybrid_results[i].get('name', 'N/A')
        
        if baseline_name != hybrid_name:
            baseline_content = baseline_results[i].get('content', 'N/A')
            hybrid_content = hybrid_results[i].get('content', 'N/A')
            different_results.append({
                'rank': i + 1,
                'baseline': baseline_content,
                'hybrid': hybrid_content
            })

    if not different_results:
        return SearchEvaluation(
            baseline_analysis="The baseline and hybrid results are identical.",
            hybrid_analysis="The hybrid and baseline results are identical.",
            comparison="There are no differences to compare.",
            baseline_score=5,
            hybrid_score=5,
            recommendation="Both methods produce the same results for this query."
        )

    prompt = f"""
    Query: "{query}"

    The following results differ between the baseline and hybrid search methods:

    {json.dumps(different_results, indent=2)}

    Please compare these differing results. Evaluate them based on relevance to the query, diversity of information, and overall quality. Provide a structured response with the following:

    1. A brief analysis of the baseline results that differ (1-2 sentences).
    2. A brief analysis of the hybrid results that differ (1-2 sentences).
    3. A comparison of the differing results (2-3 sentences), focusing on which seems more relevant or useful for the query.
    4. A score for each set of differing results on a scale of 1-10, where 10 is the best.
    5. A recommendation on which set of results is better and why, considering only these differences.

    Format your response as a JSON object with the following keys: "baseline_analysis", "hybrid_analysis", "comparison", "baseline_score", "hybrid_score", "recommendation".
    """

    response = llm.invoke(
        [
            {"role": "system", "content": "You are a search quality evaluation assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response


def compare_search_performance(queries):
    basline_total_score = 0 
    hybrid_total_score = 0

    for query in queries:
        display(HTML(f"<h2>Query: '{query}'</h2>"))
        
        baseline_result = search.baseline_search(query)
        hybrid_result = search.hybrid_search_v1(query)
        
        evaluation = get_chatgpt_evaluation(query, baseline_result, hybrid_result)
        
        basline_total_score += evaluation.baseline_score
        hybrid_total_score += evaluation.hybrid_score
        
        # Create a DataFrame for the evaluation results
        df = pd.DataFrame({
            'Metric': ['Baseline Analysis', 'Hybrid Analysis', 'Comparison', 'Baseline Score', 'Hybrid Score', 'Recommendation'],
            'Value': [
                evaluation.baseline_analysis,
                evaluation.hybrid_analysis,
                evaluation.comparison,
                f"{evaluation.baseline_score}/10",
                f"{evaluation.hybrid_score}/10",
                evaluation.recommendation
            ]
        })
        
        # Style the DataFrame
        styled_df = df.style.set_properties(**{
            'text-align': 'left',
            'white-space': 'pre-wrap',
            'max-width': '800px',
            'overflow': 'hidden',
            'text-overflow': 'ellipsis'
        }).set_table_styles([{
            'selector': 'th',
            'props': [('font-weight', 'bold'), ('text-align', 'center')]
        }])
        
        # Display the styled DataFrame
        display(styled_df)
        
        # Display the differing results
        display(HTML("<h3>Differing Results:</h3>"))
        different_results = []
        for i in range(min(5, len(baseline_result), len(hybrid_result))):
            baseline_name = baseline_result[i].get('name', 'N/A')
            hybrid_name = hybrid_result[i].get('name', 'N/A')
            if baseline_name != hybrid_name:
                different_results.append({
                    'Rank': i + 1,
                    'Baseline': baseline_name,
                    'Hybrid': hybrid_name
                })
        
        if different_results:
            diff_df = pd.DataFrame(different_results)
            styled_diff_df = diff_df.style.set_properties(**{
                'text-align': 'left',
                'white-space': 'pre-wrap',
                'max-width': '400px',
                'overflow': 'hidden',
                'text-overflow': 'ellipsis'
            }).set_table_styles([{
                'selector': 'th',
                'props': [('font-weight', 'bold'), ('text-align', 'center')]
            }])
            display(styled_diff_df)
        else:
            display(HTML("<p>No differences found in the top 5 results.</p>"))
        
        display(HTML("<hr>"))
    
    # Calculate and display total scores
    total_score = {
        'Baseline Total Score': basline_total_score,
        'Hybrid Total Score': hybrid_total_score
    }

    # Calculate the improvement percentage
    improvement = ((hybrid_total_score - basline_total_score) / basline_total_score) * 100 if basline_total_score != 0 else float('inf')

    total_score_df = pd.DataFrame(list(total_score.items()), columns=['Metric', 'Total Score'])
    styled_total_score_df = total_score_df.style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
        'max-width': '400px',
        'overflow': 'hidden',
        'text-overflow': 'ellipsis'
    }).set_table_styles([{
        'selector': 'th',
        'props': [('font-weight', 'bold'), ('text-align', 'center')]
    }])
    
    display(HTML("<h2>Total Scores</h2>"))
    display(styled_total_score_df)

    # Display the improvement percentage
    display(HTML(f"<h3>Improvement Percentage: {improvement:.2f}%</h3>"))



queries = [
    "Latest information on service development plans",
    "Recent technological changes and their impact",
    "Effective tools for team collaboration",
    "Recently updated GitHub repositories",
    "Most recent SharePoint documents",
    "Resources accessible to team managers",
    "Documents updated in 2024",
    "Frequently used category: Teams",
    "Latest papers related to research",
    "Materials for project managers",
]

# baseline score: 
# hybrid defautl score:  
# hybrid v0 score:  

compare_search_performance(queries)

,Metric,Value
0,Baseline Analysis,The baseline results are largely incoherent and lack clear relevance to the query about service development plans.
1,Hybrid Analysis,"The hybrid results, while still somewhat unclear, provide slightly more structured information and some potential relevance to service development plans."
2,Comparison,"The hybrid results are marginally better than the baseline results in terms of coherence and potential relevance. However, both sets of results are generally poor and do not directly address the query."
3,Baseline Score,2/10
4,Hybrid Score,3/10
5,Recommendation,"The hybrid results are slightly better due to their marginally improved coherence and potential relevance. However, both sets of results need significant improvement to be truly useful for the query."


,Rank,Baseline,Hybrid
0,1,Author information who.,Service work skin.
1,2,Consider development.,Stand industry development.
2,3,Surface foreign specific admit.,Consider development.
3,4,Support feeling ready.,Because high movement large.
4,5,General trial.,Author information who.


,Metric,Value
0,Baseline Analysis,The baseline results are largely incoherent and lack clear relevance to the query about recent technological changes and their impact.
1,Hybrid Analysis,"The hybrid results, while still somewhat disjointed, contain more coherent sentences and some relevant terms related to technology and impact."
2,Comparison,"The hybrid results are marginally better as they contain more coherent sentences and some relevant terms. However, both sets of results are generally poor in terms of relevance and quality."
3,Baseline Score,2/10
4,Hybrid Score,4/10
5,Recommendation,"The hybrid results are slightly better due to their marginally higher coherence and occasional relevance to the query. However, both sets of results need significant improvement to be truly useful."


,Rank,Baseline,Hybrid
0,1,Bill animal direction.,Blue air speak poor.
1,2,Artist mission back.,Policy maybe arm.
2,3,Policy maybe arm.,Republican impact.
3,4,Republican impact.,Bill animal direction.
4,5,Recent food like.,Artist mission back.


,Metric,Value
0,Baseline Analysis,The baseline results are largely incoherent and do not provide relevant information about tools for team collaboration.
1,Hybrid Analysis,The hybrid results are also incoherent and fail to address the query about effective tools for team collaboration.
2,Comparison,"Both sets of results are not relevant to the query. However, the hybrid results are slightly more structured and coherent compared to the baseline results."
3,Baseline Score,1/10
4,Hybrid Score,2/10
5,Recommendation,"Neither set of results is satisfactory for the query. However, if forced to choose, the hybrid results are marginally better due to their slightly improved coherence."


...

,Metric,Value
0,Baseline Analysis,The baseline results are largely incoherent and lack clear relevance to the query about the latest research papers.
1,Hybrid Analysis,"The hybrid results, while still somewhat disjointed, contain more coherent sentences and some references to research and related activities."
2,Comparison,"The hybrid results are marginally better as they include more relevant terms such as 'research,' 'development,' and 'study,' which align better with the query. The baseline results are mostly nonsensical and do not provide useful information."
3,Baseline Score,2/10
4,Hybrid Score,4/10
5,Recommendation,"The hybrid results are better because they contain more relevant information and coherent sentences related to research, making them more useful for the query."


,Rank,Baseline,Hybrid
0,1,Institution truth easy without leader.,Paper former protect financial.
1,2,World able also long.,Generation choose research.
2,3,Cost research blood.,Politics your politics.
3,4,Interest research answer.,Research arm group.
4,5,Generation choose research.,Interest research answer.


,Metric,Value
0,Baseline Analysis,The baseline results are largely incoherent and do not provide relevant information for project managers. They contain random phrases and lack context.
1,Hybrid Analysis,The hybrid results are also incoherent and lack relevance to the query. They consist of disjointed sentences that do not offer useful information for project managers.
2,Comparison,"Both sets of results are poor in terms of relevance and quality. However, the hybrid results are slightly more structured and contain some phrases that could be loosely related to project management. Neither set provides useful or actionable information for the query."
3,Baseline Score,2/10
4,Hybrid Score,3/10
5,Recommendation,"The hybrid results are marginally better due to slightly better structure and a few phrases that could be loosely related to project management. However, both sets of results need significant improvement in relevance and coherence."


,Rank,Baseline,Hybrid
0,2,Whose rest.,Message west.
1,3,For at hotel catch yes.,Must I.
2,4,Tv recently my.,For at hotel catch yes.
3,5,Must I.,Whose rest.


,Metric,Total Score
0,Baseline Total Score,21
1,Hybrid Total Score,31
